# Business Context

TopBank is a European banking services company, which offers financial products such as bank accounts, investment products and some types of insurance through physical agencies and an online portal. The company's main product is an ordinary bank account, in which customers can make deposits, withdrawals, and transfers.


- **Business Model:** The account is free of charge for the customer for the first 12 months as a trial period, after which, if the customer renews the service for another 12 months, a fee will be charged at the end of this period proportional to the customer's salary. The amount charged varies between 15% to 20% of the client's monthly salary. If the client's salary is lower than the average salary, the fee is 15% of the monthly salary, otherwise the fee is 20%.


- **Business Problem:** In the last few months, the number of customers canceling their accounts has been increasing, as has the company's churn rate.


- **Main Goal:** Create a high-performance prevention model to identify the probability of a customer going into churn. 


- **Secundary Goal:** Create a report on the performance and results of the model by answering the following questions::
    1. What is the current Churn rate of TopBank? How does it vary monthly if the data allows this analysis?
    2. How well does the model perform in classifying customers as churn?
    3. What is the company's revenue if it prevents customers from churning through the model?
    4. Considering a financial incentive for customers to renew for 12 more months, with a maximum of R$10,000.00 in total for all incentives. Which customers should receive the incentive and what is the value of this incentive for each customer to maximize ROI?

# 0.0. Imports

In [1]:
# data manipulation
import numpy as np
import pandas as pd
import inflection

# stratifiel sampling
from sklearn import model_selection as ms

# statistics
from scipy.stats import chi2_contingency

# machine learning models
from sklearn.ensemble import RandomForestClassifier

# model evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score

## 0.1. Auxiliar Functions

## 0.2. Load Data

- Data source: https://www.kaggle.com/datasets/mervetorkan/churndataset

In [2]:
data_raw = pd.read_csv('../datasets/churn.csv')

# convert camel case to snake case
data_raw.columns = [inflection.underscore(cols) for cols in data_raw.columns]

# data sample
data_raw.sample(10)

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
4936,4937,15792986,T'ao,580,Germany,Male,24,1,133811.78,1,1,0,17185.95,1
6770,6771,15815295,John,662,France,Female,38,2,96479.81,1,1,0,120259.41,0
7957,7958,15582149,Ts'ui,850,Germany,Female,34,3,129668.43,2,1,1,88743.99,0
4629,4630,15687153,Graham,850,Germany,Male,49,8,98649.55,1,1,0,119174.88,1
1421,1422,15742378,Swaim,520,Germany,Male,32,5,110029.77,1,1,0,56246.69,0
1554,1555,15779481,Swadling,628,France,Male,34,4,158741.43,2,1,1,126192.54,0
1160,1161,15719348,Tsao,513,France,Male,35,8,0.00,1,1,0,76640.29,1
889,890,15600462,Barwell,542,France,Female,43,8,145618.37,1,0,1,10350.74,0
9103,9104,15728362,Robertson,671,France,Female,29,3,0.00,2,1,0,158043.11,0
4727,4728,15641822,Barese,648,France,Female,19,1,0.00,2,0,1,22101.86,0


## 0.3. Split Data into Training and Test

In [3]:
data_train, data_test = ms.train_test_split(data_raw, test_size=0.2, random_state=42)

# 1.0. Data Description

In [4]:
df01 = data_train.copy()

## 1.1. Features Description

- **RowNumber:** The column number;
- **CustomerID:** Unique identifier of the customer;
- **Surname:** Last name of the customer;
- **CreditScore:** The customer's Credit Score for the consumer market;
- **Geography:** The country where the customer resides;
- **Gender:** The customer's gender;
- **Age:** The customer's age;
- **Tenure:** Number of years that the customer has been active;
- **Balance:** Monetary value that the customer has in his bank account;
- **NumOfProducts:** The number of products bought by the customer in the bank;
- **HasCrCard:** Indicates whether or not the client has a credit card;
- **IsActiveMember:** Indicates if the client has made at least one transaction in the bank account within the last 12 months;
- **EstimatedSalary:** Indicates the client's estimated monthly salary;
- **Exited:** Indicates whether or not the customer is in churn;

## 1.2. Data Types

In [5]:
df01.dtypes

row_number            int64
customer_id           int64
surname              object
credit_score          int64
geography            object
gender               object
age                   int64
tenure                int64
balance             float64
num_of_products       int64
has_cr_card           int64
is_active_member      int64
estimated_salary    float64
exited                int64
dtype: object

## 1.3. Data Dimensions

In [6]:
print(f'Number of Rows: {df01.shape[0]}')
print(f'Number of Columns: {df01.shape[1]}')

Number of Rows: 8000
Number of Columns: 14


- Relatively small database

## 1.4. Check NA

In [7]:
df01.isna().sum()

row_number          0
customer_id         0
surname             0
credit_score        0
geography           0
gender              0
age                 0
tenure              0
balance             0
num_of_products     0
has_cr_card         0
is_active_member    0
estimated_salary    0
exited              0
dtype: int64

## 1.5. Fillout NA

There is no NA in the dataset

## 1.6. Change Data Types

In [8]:
# convert customer id to string
df01['customer_id'] = df01['customer_id'].astype(object)

## 1.7. Check Balance Data

In [9]:
df01['exited'].value_counts(normalize=True)

0    0.7945
1    0.2055
Name: exited, dtype: float64

Unbalanced dataset with 80/20 of the data for the negative and positive classes, respectively.

## 1.8. Descriptive Analysis

In [10]:
# select numeric attributes
num_attributes = df01[['row_number', 'credit_score', 'age', 'tenure', 'balance', 
                       'num_of_products', 'estimated_salary']]

# select categorical and binary attributes
cat_attributes = df01[['customer_id', 'surname', 'geography', 'gender', 'has_cr_card', 
                       'is_active_member', 'exited']]

### 1.8.1. Numerical Attributes

In [11]:
# central tendency - mean, median
mean = pd.DataFrame(num_attributes.apply(np.mean)).T
median = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion tendency - std, min, max, range, skew, kurtosis
std = pd.DataFrame(num_attributes.apply(np.std)).T
min_ = pd.DataFrame(num_attributes.apply(min)).T
max_ = pd.DataFrame(num_attributes.apply(max)).T
range_ = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
skew = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
kurtosis = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# join dataframes
num_stats = pd.concat([min_, max_, range_, mean, median, skew, kurtosis]).T.reset_index()
num_stats.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'skew', 'kurtosis']

# display numerical analysis
num_stats

,attributes,min,max,range,mean,median,skew,kurtosis
0,row_number,2.00,10000.00,9998.00,5012.506875,5014.50,0.002263,-1.197591
1,credit_score,350.00,850.00,500.00,651.647625,653.00,-0.082577,-0.412964
2,age,18.00,92.00,74.00,38.897750,37.00,1.004232,1.322109
3,tenure,0.00,10.00,10.00,5.003875,5.00,0.015093,-1.150506
4,balance,0.00,250898.09,250898.09,76102.139645,96447.52,-0.129407,-1.493190
5,num_of_products,1.00,4.00,3.00,1.531375,1.00,0.721139,0.502987
6,estimated_salary,90.07,199970.74,199880.67,100431.289764,100487.72,-0.004443,-1.180224


- _credit_score_, _tenure_, _estimated_salary_ have a skew close to zero.

- _estimated_salary_ shows a below-average minimum, it is probably a system error.

### 1.8.2. Categorical Attributes

In [12]:
cat_attributes.astype(object).describe()

,customer_id,surname,geography,gender,has_cr_card,is_active_member,exited
count,8000,8000,8000,8000,8000,8000,8000
unique,8000,2618,3,2,2,2,2
top,15601116,Shih,France,Male,1,1,0
freq,1,24,3994,4362,5628,4102,6356


- There are no repeated users in the database.
- Almost half of the dataset is located in France.
- Just over half of the database is an active member.
- Database has similar numbers by gender.
- Only 3 parents present in the dataset.

# 2.0. Feature Engineering

In [13]:
df02 = df01.copy()

# 3.0. Data Filtering

In [14]:
df03 = df02.copy()

## 3.1. Filtering Rows

## 3.2. Columns Selection

- _row_number_, _customer_id_ are removed from the dataset as they do not carry information of value for learning the models.

In [15]:
df03 = df03.drop(columns=['row_number', 'customer_id'])

# 4.0. Exploratory Data Analysis

In [16]:
df04 = df03.copy()

# 5.0. Data Preparation

In [17]:
df05 = df04.copy()

## 5.1. Transformation

### 5.1.1. Encoding

In [18]:
# one hot encoding
df05 = pd.get_dummies(df05)

## 5.2. Data Preparation Check

In [19]:
# display data after preparation
df05.head().T

,9254,1561,1670,6087,6669
credit_score,686.0,632.0,559.00,561.0,517.00
age,32.0,42.0,24.00,27.0,56.00
tenure,6.0,4.0,3.00,9.0,9.00
balance,0.0,119624.6,114739.92,135637.0,142147.32
num_of_products,2.0,2.0,1.00,1.0,1.00
...,...,...,...,...,...
geography_France,1.0,0.0,0.00,1.0,1.00
geography_Germany,0.0,1.0,0.00,0.0,0.00
geography_Spain,0.0,0.0,1.00,0.0,0.00
gender_Female,0.0,0.0,0.00,1.0,0.00


# 6.0. Feature Selection

In [20]:
df06 = df05.copy()

## 6.1. Split Dataframe into Training and Validation Dataset

In [21]:
# features dataset
X = df06.drop(columns='exited')

# response dataset
y = df06['exited']

In [22]:
# split dataset into training and validation
X_train, X_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.2, random_state=42)

# 7.0. Machine Learning Model

In [23]:
# select data for machine learning models
x_train = X_train
x_val = X_val

## 7.1. Random Forest Classifier

In [24]:
model_rf = RandomForestClassifier(random_state=42).fit(x_train, y_train)

### 7.1.1. Performance

In [25]:
# prediction in validation dataset
y_rf_pred = model_rf.predict(x_val)

# accuracy
acc_rf = accuracy_score(y_val, y_rf_pred)

# precision
prec_rf = precision_score(y_val, y_rf_pred)

# recall
recall_rf = recall_score(y_val, y_rf_pred)

# performance of model
performance_rf = pd.DataFrame({'model': 'Random Forest Classifier', 
                               'accuracy': acc_rf, 
                               'precision': prec_rf, 
                               'recall': recall_rf}, 
                              index=[0])

# display performance
performance_rf

,model,accuracy,precision,recall
0,Random Forest Classifier,0.85125,0.839695,0.336391


In [26]:
# check balancing of predicition
pd.DataFrame(y_rf_pred).value_counts(normalize=True)

0    0.918125
1    0.081875
dtype: float64